In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [13]:
train_df  = pd.read_csv('../../data/mlb-player-digital-engagement-forecasting/train.csv')
teams_df = pd.read_csv('../../data/mlb-player-digital-engagement-forecasting/teams.csv')
awards_df = pd.read_csv('../../data/mlb-player-digital-engagement-forecasting/awards.csv')
players_df = pd.read_csv('../../data/mlb-player-digital-engagement-forecasting/players.csv')
seasons_df = pd.read_csv('../../data/mlb-player-digital-engagement-forecasting/seasons.csv')
ex_test_df = pd.read_csv('../../data/mlb-player-digital-engagement-forecasting/example_test.csv')
ex_submission_df = pd.read_csv('../../data/mlb-player-digital-engagement-forecasting/example_sample_submission.csv')

In [6]:
train_df.head()

,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,20180101,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,20180102,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,20180103,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,20180104,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,20180105,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN


In [15]:
train_df.shape

(1216, 12)

In [14]:
train_df.isnull().sum()

date                          0
nextDayPlayerEngagement       0
games                       577
rosters                       0
playerBoxScores             678
teamBoxScores               678
transactions                113
standings                   685
awards                      922
events                      680
playerTwitterFollowers     1176
teamTwitterFollowers       1176
dtype: int64

In [7]:
teams_df.head()

,id,name,teamName,teamCode,shortName,abbreviation,locationName,leagueId,leagueName,divisionId,divisionName,venueId,venueName
0,108,Los Angeles Angels,Angels,ana,LA Angels,LAA,Anaheim,103,American League,200,American League West,1,Angel Stadium
1,109,Arizona Diamondbacks,D-backs,ari,Arizona,ARI,Phoenix,104,National League,203,National League West,15,Chase Field
2,110,Baltimore Orioles,Orioles,bal,Baltimore,BAL,Baltimore,103,American League,201,American League East,2,Oriole Park at Camden Yards
3,111,Boston Red Sox,Red Sox,bos,Boston,BOS,Boston,103,American League,201,American League East,3,Fenway Park
4,112,Chicago Cubs,Cubs,chn,Chi Cubs,CHC,Chicago,104,National League,205,National League Central,17,Wrigley Field


In [8]:
awards_df.head()

,awardDate,awardSeason,awardId,awardName,playerId,playerName,awardPlayerTeamId
0,2017-12-21,2017,WARRENSPAHN,Warren Spahn Award,477132,Clayton Kershaw,119.0
1,2017-12-20,2017,MILBORGAS,MiLB.com Organization All-Star,474319,Brandon Snyder,120.0
2,2017-12-20,2017,MILBORGAS,MiLB.com Organization All-Star,592530,Jose Marmolejos,120.0
3,2017-12-20,2017,MILBORGAS,MiLB.com Organization All-Star,593833,Wander Suero,120.0
4,2017-12-20,2017,MILBORGAS,MiLB.com Organization All-Star,600466,Raudy Read,120.0


In [16]:
min(awards_df['awardDate'])

'1998-06-21'

In [17]:
max(awards_df['awardDate'])

'2017-12-21'

In [9]:
payers_df.head()

,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,665482,Gilberto Celestino,1999-02-13,2021-06-02,Santo Domingo,NaN,Dominican Republic,72,170,8,Outfielder,False
1,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True
2,661269,Vladimir Gutierrez,1995-09-18,2021-05-28,Havana,NaN,Cuba,73,190,1,Pitcher,True
3,669212,Eli Morgan,1996-05-13,2021-05-28,Rancho Palos Verdes,CA,USA,70,190,1,Pitcher,True
4,666201,Alek Manoah,1998-01-09,2021-05-27,Homestead,FL,USA,78,260,1,Pitcher,True


In [10]:
seasons_df.head()

,seasonId,seasonStartDate,seasonEndDate,preSeasonStartDate,preSeasonEndDate,regularSeasonStartDate,regularSeasonEndDate,lastDate1stHalf,allStarDate,firstDate2ndHalf,postSeasonStartDate,postSeasonEndDate
0,2017,2017-04-02,2017-11-01,2017-02-22,2017-04-01,2017-04-02,2017-10-01,2017-07-09,2017-07-11,2017-07-14,2017-10-03,2017-11-01
1,2018,2018-03-29,2018-10-28,2018-02-21,2018-03-27,2018-03-29,2018-10-01,2018-07-15,2018-07-17,2018-07-19,2018-10-02,2018-10-28
2,2019,2019-03-20,2019-10-30,2019-02-21,2019-03-26,2019-03-20,2019-09-29,2019-07-07,2019-07-09,2019-07-11,2019-10-01,2019-10-30
3,2020,2020-07-23,2020-10-28,2020-02-21,2020-07-22,2020-07-23,2020-09-27,2020-08-25,NaN,2020-08-26,2020-09-29,2020-10-28
4,2021,2021-02-28,2021-10-31,2021-02-28,2021-03-30,2021-04-01,2021-10-03,2021-07-11,2021-07-13,2021-07-15,2021-10-04,2021-10-31


In [11]:
ex_test_df.head()

,date,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,20210426,"[{""gamePk"":634374,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-26"",""t...","[{""home"":1,""gamePk"":634377,""gameDate"":""2021-04...","[{""home"":1,""teamId"":139,""gamePk"":634343,""gameD...","[{""transactionId"":480386,""playerId"":543685,""pl...","[{""season"":2021,""gameDate"":""2021-04-26"",""divis...",NaN,"[{""gamePk"":634433,""gameDate"":""2021-04-26"",""gam...",NaN,NaN
1,20210427,"[{""gamePk"":634318,""gameType"":""R"",""season"":2021...","[{""playerId"":443558,""gameDate"":""2021-04-27"",""t...","[{""home"":1,""gamePk"":634320,""gameDate"":""2021-04...","[{""home"":1,""teamId"":117,""gamePk"":634333,""gameD...","[{""transactionId"":480456,""playerId"":642162,""pl...","[{""season"":2021,""gameDate"":""2021-04-27"",""divis...",NaN,"[{""gamePk"":634332,""gameDate"":""2021-04-27"",""gam...",NaN,NaN
2,20210428,"[{""gamePk"":634309,""gameType"":""R"",""season"":2021...","[{""playerId"":429722,""gameDate"":""2021-04-28"",""t...","[{""home"":1,""gamePk"":634310,""gameDate"":""2021-04...","[{""home"":0,""teamId"":111,""gamePk"":634310,""gameD...","[{""transactionId"":480728,""playerId"":545358,""pl...","[{""season"":2021,""gameDate"":""2021-04-28"",""divis...",NaN,"[{""gamePk"":634317,""gameDate"":""2021-04-28"",""gam...",NaN,NaN
3,20210429,"[{""gamePk"":634330,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-29"",""t...","[{""home"":1,""gamePk"":634330,""gameDate"":""2021-04...","[{""home"":0,""teamId"":119,""gamePk"":634346,""gameD...","[{""transactionId"":480993,""playerId"":606965,""pl...","[{""season"":2021,""gameDate"":""2021-04-29"",""divis...",NaN,"[{""gamePk"":634346,""gameDate"":""2021-04-29"",""gam...",NaN,NaN
4,20210430,"[{""gamePk"":634287,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-30"",""t...","[{""home"":1,""gamePk"":634305,""gameDate"":""2021-04...","[{""home"":1,""teamId"":135,""gamePk"":634303,""gameD...",NaN,"[{""season"":2021,""gameDate"":""2021-04-30"",""divis...","[{""awardId"":""NLRRELMON"",""awardName"":""NL Reliev...","[{""gamePk"":634327,""gameDate"":""2021-04-30"",""gam...",NaN,NaN


In [12]:
ex_submission_df.head()

,date,date_playerId,target1,target2,target3,target4
0,20210426,20210427_656669,0,0,0,0
1,20210426,20210427_543475,0,0,0,0
2,20210426,20210427_592866,0,0,0,0
3,20210426,20210427_452678,0,0,0,0
4,20210426,20210427_570257,0,0,0,0


## Train Data Analysis

In [25]:
train_df.shape

(1216, 12)

In [26]:
train_df['date'] = pd.to_datetime(train_df['date'], format = "%Y%m%d")

In [27]:
train_df.head()

,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,2018-01-01,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,2018-01-02,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,2018-01-03,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,2018-01-04,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,2018-01-05,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN


In [31]:
min(train_df['date'])

Timestamp('2018-01-01 00:00:00')

In [32]:
max(train_df['date'])

Timestamp('2021-04-30 00:00:00')

In [28]:
train_df.isnull().sum()

date                          0
nextDayPlayerEngagement       0
games                       577
rosters                       0
playerBoxScores             678
teamBoxScores               678
transactions                113
standings                   685
awards                      922
events                      680
playerTwitterFollowers     1176
teamTwitterFollowers       1176
dtype: int64

In [37]:
train_df['playerBoxScores']

0                                                     NaN
1                                                     NaN
2                                                     NaN
3                                                     NaN
4                                                     NaN
                              ...                        
1211    [{"home":1,"gamePk":634377,"gameDate":"2021-04...
1212    [{"home":1,"gamePk":634320,"gameDate":"2021-04...
1213    [{"home":1,"gamePk":634310,"gameDate":"2021-04...
1214    [{"home":1,"gamePk":634330,"gameDate":"2021-04...
1215    [{"home":1,"gamePk":634305,"gameDate":"2021-04...
Name: playerBoxScores, Length: 1216, dtype: object

In [34]:
train_df.iloc[0]['nextDayPlayerEngagement']

'[{"engagementMetricsDate":"2018-01-02","playerId":628317,"target1":0.011167070542384616,"target2":4.4747081712062258,"target3":0.0051677297424994094,"target4":5.7352941176470589},{"engagementMetricsDate":"2018-01-02","playerId":547989,"target1":0.042993221588180773,"target2":5.5933852140077827,"target3":0.045033073470351993,"target4":2.7941176470588238},{"engagementMetricsDate":"2018-01-02","playerId":519317,"target1":0.97432690482305784,"target2":56.177042801556418,"target3":13.693745570517363,"target4":64.166666666666671},{"engagementMetricsDate":"2018-01-02","playerId":607625,"target1":0.0067002423254307695,"target2":2.6750972762645913,"target3":0.0051677297424994094,"target4":1.8627450980392157},{"engagementMetricsDate":"2018-01-02","playerId":592547,"target1":0.0011167070542384616,"target2":0.632295719844358,"target3":0.0029529884242853769,"target4":0.93137254901960786},{"engagementMetricsDate":"2018-01-02","playerId":641553,"target1":0.011725424069503847,"target2":3.842412451361

In [36]:
train_df.iloc[1]['rosters']

'[{"playerId":134181,"gameDate":"2018-01-02","teamId":140,"statusCode":"A","status":"Active"},{"playerId":425794,"gameDate":"2018-01-02","teamId":138,"statusCode":"A","status":"Active"},{"playerId":429664,"gameDate":"2018-01-02","teamId":136,"statusCode":"A","status":"Active"},{"playerId":430935,"gameDate":"2018-01-02","teamId":140,"statusCode":"A","status":"Active"},{"playerId":434628,"gameDate":"2018-01-02","teamId":118,"statusCode":"A","status":"Active"},{"playerId":444482,"gameDate":"2018-01-02","teamId":109,"statusCode":"A","status":"Active"},{"playerId":446899,"gameDate":"2018-01-02","teamId":146,"statusCode":"A","status":"Active"},{"playerId":452657,"gameDate":"2018-01-02","teamId":112,"statusCode":"A","status":"Active"},{"playerId":453172,"gameDate":"2018-01-02","teamId":113,"statusCode":"A","status":"Active"},{"playerId":453178,"gameDate":"2018-01-02","teamId":118,"statusCode":"A","status":"Active"},{"playerId":453343,"gameDate":"2018-01-02","teamId":137,"statusCode":"A","stat